## Exceptions Report

In [ ]:
'''
# Files needed: 
#   UPDATE EACH TIME YOU RUN THE REPORT
#       - SupplyDemand.csv = Current daily MRP report exported as csv to U:\foldername1\foldername2\Input_Data\SupplyDemand.csv
#       - Inventory.csv = Current daily inventory from Discoverer exported as csv to U:\foldername1\foldername2\Input_Data\Inventory.csv
#               # CODE REMOVED NONNETTABLE LOCATIONS, UPDATE CODE IF NEW SUBINVENTORIES ARE CREATED

#   UPDATE WHEN CHANGES ARE MADE IN ORACLE
#       - PLY_Item.csv = Current item details report from Discoverer exported as csv to U:\foldername1\foldername2\Input_Data\PLY_Item.csv
#           - SCE- Item Detail Listing w Item Status
#       - current master.csv reference file 


# Exception_Metrics.csv = file that gets metrics updates every time this script is run



#If you need to update encoding
#with open('filename.csv') as f:
#    print(f)

'''

In [ ]:
#Import modules

from os import chdir, system
import datetime
import pandas as pd

In [ ]:
# Set the input file directory - ALL IMPORTED FILES COME FROM THE PATH.
# IF YOU NEED TO, ADD A NEW PATH ABOVE A FILE THATS IMPORTED FROM A DIFFERENT FOLDER.
chdir(r'U:\foldername1\foldername2\Input_Data')

In [ ]:
#Import Supply Demand

# Read in the current Supply Demand file
data = pd.read_csv('SupplyDemand.csv', encoding='utf-8',
                   usecols=['Item','Order Type','Action','Planner','Using Assembly',
                            'Source Supplier', 'Sugg Dock Date', 'Old Dock Date',
                            'Sugg Due Date','Sugg Order Date','Order Number','Qty/Rate'],
                      dtype={
                             'Old Dock Date' : str,                #07   * changed to str
                             'Sugg Order Date' : str,              #08   * changed to str
                             'Sugg Dock Date' : str,               #14   * changed to str
                             'Source Supplier' : str,              #15   * changed to str
                             }
                   )

print("data file is loaded")

#preview the supply demand dataframe
data.head()

In [ ]:

#If you need to update encoding
with open('Inventory.csv') as f:
    print(f)

#import inventory report
# skiprows=4 allows you to drop the first four rows (originally blank when you pull the report)
inventory = pd.read_csv('Inventory.csv', skiprows=4, encoding='latin1', usecols=['Subinventory', 'Item', 'Quantity'])

# List of non-nettable subinventory locations
# These we do not want included in inventory
non_nettable_subinventories = ['FG_ENGHLD', 'MRB FG', 'MRB-RAW', 'MRB-WIP', 'PROJECTS', 'RAW_ENGHLD', 'WIP_ENGHLD']

#drop the rows that have non-nettable locations
inventory = inventory[~inventory.Subinventory.isin(non_nettable_subinventories)]

#confirm the drop by printing unique subinventory locations
print('List of unique subinventory locations:')
print(inventory['Subinventory'].unique())

#drop the subinventory column
inventory.drop('Subinventory', axis=1, inplace=True)

#preview the inventory df
inventory.head()


In [ ]:
#Import PLY Items to get buyer names

# Read in the current Item details file
buyers = pd.read_csv('PLY_Items.csv', encoding='utf-8',
                     usecols=['Item', 'Buyer', 'Planner Code', 'Inventory Planning Code'])

# Change the buyers' names to just first names
buyers['Buyer'] = buyers['Buyer'].map({'lastname1, firstname1' : 'firstname1', 'lastname2, firstname2' : 'firstname2', 'lastname3, firstname3' : 'firstname3'})

print("item_data file is loaded")

#preview the buyer df
buyers.head()

In [ ]:
#Import safety stock

# Load the frozen cost from master
master = pd.read_csv('master.csv', encoding='cp1252', usecols=['Item', 'Frozen Cost', 'Safety Stock'])
master.rename(columns={'Safety Stock':'Safety Stock Qty'}, inplace=True)

#preview the master df
master.head()


In [ ]:
#Start prep main frame

# Add the buyer's name column to the data dataframe
data = pd.merge(data, buyers, on='Item', how='left')

#preview the merged df
data.head()

In [ ]:
#Remove min-max items

minmax = data['Inventory Planning Code'] == "Min-max planning"
data = data[~minmax]

# Fact check - only Not Planned should exist
print('List of unique Inventory Planning Code:')
print(data['Inventory Planning Code'].unique())

# Delete planning code column
del data['Inventory Planning Code']


In [ ]:
#Add planner code for using assembly

del data['Planner Code'] # need to remove this and remerge

del buyers['Buyer'] # don't need this anymore - already merged

buyers.rename(columns={'Item':'Using Assembly', 'Planner Code':'Planner Code for Using Assembly'},
              inplace=True)

data = pd.merge(data, buyers, how='left', on='Using Assembly')

#preview the data df
data.head()

In [ ]:
# Preview datatypes for each column
display(data.dtypes)

In [ ]:
#Change date formats
# make sure all the dates are the same datetime format for sorting

data['Sugg Dock Date'] = pd.to_datetime(data['Sugg Dock Date'])
data['Old Dock Date'] = pd.to_datetime(data['Old Dock Date'])
data['Sugg Due Date'] = pd.to_datetime(data['Sugg Due Date'])
data['Sugg Order Date'] = pd.to_datetime(data['Sugg Order Date'])

In [ ]:
# Preview datatypes for each column to confirm the change
display(data.dtypes)

In [ ]:
#Sort by Planner, Item, Dates

# IF Dock Date is empty, fill it with Sugg Due Date
data['Old Dock Date'].fillna(data['Sugg Due Date'].dt.date, inplace=True)

# Put all of the items in order by date
data = data.sort_values(['Planner','Item', 'Old Dock Date', 'Sugg Due Date'])

data.head()

In [ ]:
#Calculate qty on hand

# Calculate the current inventory column
qty_on_hand = inventory.groupby(['Item'])['Quantity'].sum()

data['CumSum'] = data.groupby(['Item'])['Qty/Rate'].apply(lambda x: x.cumsum())

data = pd.merge(data, qty_on_hand, on='Item', how='left')

data['Quantity'] = data['Quantity'].fillna(0)

data['Current INV'] = data['CumSum'] + data['Quantity']

In [ ]:
#Add comments column
data['Comments'] = ""

# Add safety stock and future SS columns to match the excel file.
data['Future SS'] = ""

In [ ]:
#Diff in Days

# Add a column for the difference in days between old date and new
#      adding .dt.days converts it to an integer (number of days)
#      this allows us to add the 'reschedule over 90' feature later on
data['Diff in Days'] = (data['Old Dock Date'] - data['Sugg Dock Date']).dt.days

In [ ]:
#Compute Extended cost

data = data.merge(master, how='left', on='Item')

data['Order Cost'] = data['Qty/Rate'] * data['Frozen Cost']

data.drop(columns=['Frozen Cost'], inplace=True)

#Remove negatives in Order Cost

neg = data['Order Cost'] <= 0
data.loc[neg, 'Order Cost'] = ""

In [ ]:
#Put columns in order

# Set the order of the columns we want
data = data[['Item','Comments','Order Type','Action','Planner','Buyer','Using Assembly',
             'Planner Code for Using Assembly','Source Supplier','Diff in Days', 'Sugg Dock Date',
             'Old Dock Date','Sugg Due Date','Sugg Order Date','Order Number','Qty/Rate',
             'Current INV', 'Safety Stock Qty', 'Future SS', 'Order Cost']]
#preview
data.head()

In [ ]:
#List = items to look into
# This sections filters down the rows to the items we want to look at

cancels = data['Action'] == "Cancel"
outs = data['Action'] == "Reschedule Out"
ins = data['Action'] == "Reschedule In"

cancel_or_out = (cancels | outs) # & in_indents
items_look_into = data[(cancel_or_out | ins)]

purch = items_look_into['Order Type'] == 'Purchase order'

items_look_into = items_look_into[purch]

# final list with rows narrowed down to any item with rescheduled ins
# or item with yes for Indented BOM and either cancel or rescheduled outs
df = data[data['Item'].isin(items_look_into['Item'])]

df.head()

In [ ]:
#Make the Num col

# Creates a column "Num" with a single integer for each material in the list
items_list = items_look_into['Item'].drop_duplicates()
items_list.reset_index(drop=True, inplace=True)
df3 = pd.Series.to_frame(items_list)
df3['Num'] = range(1, len(df3.index)+1)
df = pd.merge(df, df3, on='Item', how='left')

df.head()

In [ ]:
#Add line cnt column
# TO SEE THE PURPOSE OF THIS LINE, COMMENT OUT 'del df['lines']' LATER IN THE CODE.
# YOU CAN SEE HOW ITEMS ARE GROUPED AND NUMBERED

df['lines'] = 1

for cnt in (df.index+1):
    try:
        if df.Item[cnt-1] == df.Item[cnt]:
            df.at[cnt, 'lines'] += df.at[cnt-1, 'lines']
    except (KeyError):
        pass

In [ ]:
#LC Add columns for coverage


print("Adding test and covered columns")
df['test'] = 0.0
df['covered'] = "Yes"
df['last'] = ""

#If current inv is less than 0, covered = No
for x in df.index:
    if df['Current INV'][x] < 0:
        df.at[x, 'covered'] = 'No'
        
print("test and covered columns done")

print("Marking Okay, Not Okay, Reschedule Over 90 and Past Due")

## SAME DAY - POs that arrive the same day as a negative demand are marked 'OK'


for x in df.index:
    try:
        if df.Action[x] == 'Reschedule In':   # Isolate reschedule ins only
            if df.at[x, 'Sugg Due Date'] >= df.at[x, 'Old Dock Date']:   #if Sug due date is greater than or the same as old dock date
                df.at[x, 'Comments'] = 'OK'   #Mark OK
            elif df['lines'][x] == 1 and df.at[x, 'Current INV'] >= 0:
                df.at[x, 'Comments'] = 'OK'
            elif df['lines'][x] > 1 and df.at[x-1, 'Current INV'] >= 0: #if it's not the first item in that list and the line prior is not negative (or 0), mark OK
                df.at[x, 'Comments'] = 'OK'
            else:   #The rest are not ok
                df.at[x, 'Comments'] = 'Not OK'
    except(KeyError):
        pass

# Mark Reschedule Outs over 90
for x in df.index:
    try:
        if df.Action[x] == 'Reschedule Out':
            if df.at[x, 'Diff in Days'] <= -90:
                df.at[x, 'Comments'] = 'Reschedule Over 90'
    except(KeyError):
        pass

# Mark Past Due POs
# This was not working when I used df.at[x, 'Order Type'] == 'Purchase Order'
# So I switched it to go off of when qty/rate > 0 

#get yesterdays date to determine past due POs
from datetime import date, timedelta
yesterday = date.today() - timedelta(days=1)


for x in df.index:
    try:
        if df.at[x, 'Qty/Rate'] > 0:
            if df.at[x, 'Old Dock Date'] < yesterday: #pd.to_datetime('today'):
                df.at[x, 'Comments'] = 'Past Due PO'
    except(KeyError):
        pass


In [ ]:
#Calculate metrics

total_cancels = df[df['Action'] == "Cancel" ]
total_ins = df[df['Action'] == "Reschedule In"]
total_po_outs = df[(df['Action'] == "Reschedule Out") & (df['Order Type'] == 'Purchase order')]
total_wo_outs = df[(df['Action'] == "Reschedule Out") & (df['Order Type'] == 'Work order')]

metric_cancels = total_cancels['Item'].count()
metric_ins = total_ins['Item'].count()
metric_po_outs = total_po_outs['Item'].count()
metric_wo_outs = total_wo_outs['Item'].count()

In [ ]:
#Print metrics

print("Reschedule Ins = ", metric_ins)
print("Purchase order reschedule outs = ", metric_po_outs)
print("Work order reschedule outs = ", metric_wo_outs)
print("Cancels = ", metric_cancels)

In [ ]:
#fill blanks upwards

df['Using Assembly'].bfill(inplace=True)
df['Planner Code for Using Assembly'].bfill(inplace=True)

#del lines column
del df['lines']

In [ ]:
#Open previous report

# Open the previous report to get the comments from last time
chdir(r'U:\foldername3\foldername4')

previous = pd.read_excel("Exceptions.xlsm", skiprows=3, 
                         usecols=['Item', 'Last Week', 'Comments', 'Order Number', 'Qty/Rate'])

In [ ]:
#Copy Last Week to Comments

# Loop through each line, copy Last Week to Comments if it is blank
for x in previous.index:
    if pd.isna(previous.Comments[x]):
        previous.at[x, 'Comments'] = previous.at[x, 'Last Week']

#Change Comments column to Last Week

del previous['Last Week']
previous.rename(columns={'Comments':'Last Week'}, inplace=True)

In [ ]:
#Remove duplicates

# keep only rows that have comments
previous.dropna(axis=0, how='any', inplace=True)

previous.drop_duplicates(['Item', 'Order Number', 'Qty/Rate'], inplace=True)

In [ ]:
#Merge previous with df

# merge on 3 columns: Item, Order Number, Qty/Rate
df = df.merge(previous, how='left', on=['Item', 'Order Number', 'Qty/Rate'])

In [ ]:
#Move Last Week column

# copy Last Week column
temp = df['Last Week']

# delete Last Week column
del df['Last Week']

# insert Last Week column between Item and Comments
df.insert(1, "Last Week", temp)


### This changes the output to YYYY-MM-DD
df['Sugg Dock Date'] = pd.to_datetime(df['Sugg Dock Date']).dt.date

In [ ]:
#Set the output folder

chdir(r'U:\foldername1\foldername2\Output_Data')
filename = "Exceptions_output.xlsx"

print("ready to save the output file and open it with excel.....")

In [ ]:
#Write out sheets

print("Writing everything to a spreadsheet file")

writer = pd.ExcelWriter(filename, engine='xlsxwriter') # pylint: disable=abstract-class-instantiated

# add lines like this for each sheet to write out
df.to_excel(writer, sheet_name='Sheet1')

# future_ss.to_excel(writer, sheet_name='Future SS')

# Save the whole thing
writer.save()
print("file saved, opening excel")

In [ ]:
#Open excel

# Opens the file in excel
system("start excel " + filename)